In [14]:
# ==== LOAD MODEL TỪ DATASET & CHẠY SUY LUẬN ====

import json
import pathlib
import shutil
from typing import Any, Dict, Optional

import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForMaskedLM
import unicodedata

DEFAULT_MODEL_NAME = 'vinai/phobert-base'
DEFAULT_MAX_SEQ_LEN = 256
DETECTOR_AGG = 'mean'  # mean|max: mean = AI phải chiếm tỷ lệ đáng kể trong toàn bài
CHUNK_STRIDE_TOKENS = 128  # khớp Train.py (CHUNK_STRIDE)
DETECTOR_MAX_WINDOWS = None  # khớp Train.py: không cap số cửa sổ (lấy hết chunks)
COMBINED_THRESHOLD = 0.5  # nhãn cho combined_prob_ai

if 'IS_KAGGLE' not in globals():
    IS_KAGGLE = pathlib.Path('/kaggle/input').exists()
if 'ROOT' not in globals():
    ROOT = pathlib.Path('/kaggle/working') if IS_KAGGLE else pathlib.Path.cwd()
if 'ARTIFACT_DIR' not in globals():
    ARTIFACT_DIR = ROOT / 'artifacts'
if 'DEVICE' not in globals():
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if 'MODEL_NAME' not in globals():
    MODEL_NAME = DEFAULT_MODEL_NAME
if 'tokenizer' not in globals():
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if 'MAX_SEQ_LEN' not in globals():
    MAX_SEQ_LEN = min(DEFAULT_MAX_SEQ_LEN, getattr(tokenizer, 'model_max_length', DEFAULT_MAX_SEQ_LEN) or DEFAULT_MAX_SEQ_LEN)

import regex as re
SENT_SPLIT = re.compile(r'(?<=[.!?…])\s+')
WORD_RE = re.compile(r"\p{L}+(?:['’]\p{L}+)?", re.UNICODE)


# ---- Text normalization (giúp ổn định output; tránh unicode/space làm flip) ----

def normalize_text(text: str) -> str:
    cleaned = unicodedata.normalize('NFC', text or '')
    transl_table = str.maketrans({'“': '"', '”': '"', '’': "'", '–': '-', '—': '-'})
    cleaned = cleaned.translate(transl_table)
    cleaned = re.sub(r'\s+', ' ', cleaned)
    return cleaned.strip()


# ---- Optional: MaskedLM pseudo-perplexity model (nhỏ, dùng làm feature) ----

import math
import zlib
from collections import Counter

PPL_MODEL_NAME = 'vinai/phobert-base'
PPL_MAX_TOKENS = 96        # cap tokens để không quá chậm
PPL_STEP = 6              # mask mỗi k token (lấy mẫu)
PPL_MAX_POSITIONS = 24    # cap số vị trí mask

# Token-level advanced features
TOKEN_FULL_MAX_CHARS = 8000  # tránh tokenize full text quá dài (chậm)

if 'mlm_model' not in globals():
    mlm_model = None


def load_mlm_model(model_name: str = PPL_MODEL_NAME):
    global mlm_model
    if mlm_model is not None:
        return mlm_model
    try:
        m = AutoModelForMaskedLM.from_pretrained(model_name).to(DEVICE)
        m.eval()
        mlm_model = m
        print('✔️ Đã load MaskedLM cho pseudo-perplexity:', model_name)
        return mlm_model
    except Exception as e:
        print('⚠️ Không load được MaskedLM (pseudo-perplexity). Lý do:', repr(e))
        print('   Gợi ý: kiểm tra internet/cache HF hoặc đổi PPL_MODEL_NAME.')
        mlm_model = None
        return None


def maskedlm_pseudo_ppl(text: str) -> float:
    # Pseudo-perplexity cho MaskedLM: mask token (lấy mẫu) và tính -log P(token|context).
    if mlm_model is None:
        return 0.0
    if tokenizer.mask_token_id is None:
        return 0.0
    clean = normalize_text(text)
    if not clean:
        return 0.0

    enc = tokenizer(clean, return_tensors='pt', truncation=True, max_length=PPL_MAX_TOKENS)
    input_ids = enc['input_ids'][0].to(DEVICE)
    attn = enc.get('attention_mask', None)
    if attn is None:
        attn = torch.ones_like(input_ids)
    else:
        attn = attn[0].to(DEVICE)

    special = set(getattr(tokenizer, 'all_special_ids', []) or [])
    valid_positions = []
    for i, (tid, m) in enumerate(zip(input_ids.tolist(), attn.tolist())):
        if m != 1:
            continue
        if tid in special:
            continue
        valid_positions.append(i)
    if not valid_positions:
        return 0.0

    positions = valid_positions[::max(1, int(PPL_STEP))]
    positions = positions[:int(PPL_MAX_POSITIONS)]
    if not positions:
        return 0.0

    total_nll = 0.0
    n = 0
    with torch.inference_mode():
        for pos in positions:
            masked_ids = input_ids.clone()
            true_id = masked_ids[pos].item()
            masked_ids[pos] = tokenizer.mask_token_id
            out = mlm_model(input_ids=masked_ids.unsqueeze(0), attention_mask=attn.unsqueeze(0))
            logits = out.logits[0, pos]
            logp = torch.log_softmax(logits, dim=-1)[true_id].item()
            total_nll += -logp
            n += 1
    nll = total_nll / max(n, 1)
    ppl = math.exp(min(nll, 20.0))
    return float(ppl)


# ---- Model prediction ----


def _predict_prob_from_ids(input_ids_1d: list, model) -> float:
    ids = torch.tensor([input_ids_1d], device=DEVICE)
    attn = torch.ones_like(ids)
    with torch.inference_mode():
        logits = model(input_ids=ids, attention_mask=attn).logits
        prob_ai = torch.softmax(logits, dim=-1)[0, 1].item()
    return float(prob_ai)


def detector_predict(
    text: str,
    model,
    threshold: float = 0.5,
    agg: str = 'mean',
    stride_tokens: Optional[int] = None,
    stride_ratio: float = 0.5,
    max_windows: Optional[int] = None,
 ) -> Dict[str, Any]:
    """Predict trên *toàn văn* bằng cách chạy nhiều cửa sổ token (nếu text dài).

    - `agg='mean'` ổn định hơn (phương án B: AI phải chiếm tỷ lệ đáng kể).
    - `agg='max'` nhạy với 1 đoạn AI mạnh.
    - Ưu tiên `stride_tokens` để đồng bộ với Train.py; nếu None thì dùng `stride_ratio`.
    - `max_windows=None` để khớp Train.py (không cap số chunk/cửa sổ).

    Trả về đúng 1 `prob_ai` (không trả highlight).
    """
    clean_text = normalize_text(text)
    if not clean_text:
        return {'prob_ai': 0.0, 'label': 'Human', 'threshold': threshold}

    try:
        ids_no_special = (tokenizer(clean_text, add_special_tokens=False, truncation=False).get('input_ids', []) or [])
    except Exception:
        # fallback: nếu tokenize full lỗi thì dùng truncate như cũ
        inputs = tokenizer(clean_text, return_tensors='pt', truncation=True, max_length=MAX_SEQ_LEN).to(DEVICE)
        with torch.inference_mode():
            logits = model(**inputs).logits
            prob_ai = torch.softmax(logits, dim=-1)[0, 1].item()
        prob_ai_out = float(prob_ai)
        label = 'AI' if prob_ai_out >= threshold else 'Human'
        return {'prob_ai': prob_ai_out, 'label': label, 'threshold': threshold}

    n_special = int(getattr(tokenizer, 'num_special_tokens_to_add', lambda pair=False: 2)(pair=False) or 2)
    win_len = max(8, int(MAX_SEQ_LEN) - n_special)

    probs = None
    if len(ids_no_special) <= win_len:
        ids = tokenizer.build_inputs_with_special_tokens(ids_no_special)
        prob_ai = _predict_prob_from_ids(ids, model)
    else:
        if stride_tokens is not None:
            stride = max(1, int(stride_tokens))
        else:
            stride = max(1, int(win_len * float(stride_ratio)))
        stride = min(stride, win_len)

        probs = []
        for start in range(0, len(ids_no_special), stride):
            chunk = ids_no_special[start:start + win_len]
            if len(chunk) < 8:
                break
            ids = tokenizer.build_inputs_with_special_tokens(chunk)
            probs.append(_predict_prob_from_ids(ids, model))
            if start + win_len >= len(ids_no_special):
                break
            if max_windows is not None and len(probs) >= int(max_windows):
                break

        if not probs:
            ids = tokenizer(clean_text, add_special_tokens=True, truncation=True, max_length=MAX_SEQ_LEN).get('input_ids', [])
            prob_ai = _predict_prob_from_ids(ids, model)
        else:
            mean_v = float(np.mean(probs))
            max_v = float(np.max(probs))
            if agg == 'mean':
                prob_ai = mean_v
            else:
                prob_ai = max_v

    prob_ai_out = float(prob_ai)
    label = 'AI' if prob_ai_out >= threshold else 'Human'
    out = {'prob_ai': prob_ai_out, 'label': label, 'threshold': threshold}
    if probs:
        out['n_windows'] = int(len(probs))
        out['prob_ai_mean'] = float(np.mean(probs))
        out['prob_ai_max'] = float(np.max(probs))
    return out


# ---- Extra features (thuộc tính văn bản) ----

# Regex helpers
PUNCT_RE = re.compile(r"[\p{P}\p{S}]", re.UNICODE)
DIGIT_RE = re.compile(r"\p{N}", re.UNICODE)
LETTER_RE = re.compile(r"\p{L}", re.UNICODE)
UPPER_RE = re.compile(r"\p{Lu}", re.UNICODE)
URL_RE = re.compile(r"https?://\S+|www\.\S+", re.IGNORECASE)
EMAIL_RE = re.compile(r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}", re.IGNORECASE)
QUOTE_RE = re.compile(r"['\"“”‘’]", re.UNICODE)
BULLET_LINE_RE = re.compile(r"(?m)^\s*(?:[-*•]|\d+[\).]|[a-zA-Z][\).])\s+", re.UNICODE)

# Stopwords (nhỏ/gọn)
VI_STOPWORDS = {
    'và','là','của','cho','một','những','các','đã','đang','sẽ','với','trong','khi','để','này','đó','ở',
    'tôi','bạn','anh','chị','em','chúng','chúng tôi','chúng ta','họ','nó','thì','như','vì','nên',
    'cũng','rất','không','có','đến','từ','ra','vào','lại','hay','được','bị','vẫn','nhiều','ít'
}


def _words(s: str):
    return WORD_RE.findall(s.lower() if isinstance(s, str) else '')


def compute_text_features(text: str) -> Dict[str, float]:
    raw = text or ''
    t = normalize_text(raw)
    chars = len(t)
    raw_chars = len(raw)

    punct = len(PUNCT_RE.findall(t))
    digits = len(DIGIT_RE.findall(t))

    url_count = len(URL_RE.findall(raw))
    email_count = len(EMAIL_RE.findall(raw))
    quote_count = len(QUOTE_RE.findall(raw))
    bullet_lines = len(BULLET_LINE_RE.findall(raw))
    newline_count = raw.count('\n')

    letters = len(LETTER_RE.findall(t))
    uppers = len(UPPER_RE.findall(t))
    uppercase_ratio = (uppers / (letters + 1e-6)) if letters else 0.0

    words = _words(t)
    n_words = len(words)
    uniq = len(set(words)) if n_words else 0
    uniq_ratio = (uniq / n_words) if n_words else 0.0
    avg_word_len = (sum(len(w) for w in words) / n_words) if n_words else 0.0

    sents = [s.strip() for s in SENT_SPLIT.split(t) if s.strip()]
    sent_lens = [len(_words(s)) for s in sents] if sents else []
    n_sents = len(sent_lens)
    avg_sent_len = (sum(sent_lens) / n_sents) if n_sents else 0.0
    var_sent = (sum((x - avg_sent_len) ** 2 for x in sent_lens) / n_sents) if n_sents else 0.0
    std_sent = math.sqrt(var_sent) if n_sents else 0.0
    burstiness = (std_sent / (avg_sent_len + 1e-6)) if n_sents else 0.0

    paras = [p.strip() for p in re.split(r"\n\s*\n+", raw) if p.strip()]
    n_paras = len(paras) if paras else 0
    para_lens = [len(_words(normalize_text(p))) for p in paras] if paras else []
    avg_para_len = (sum(para_lens) / n_paras) if n_paras else 0.0

    max_rep = 0.0
    hapax_ratio = 0.0
    entropy = 0.0
    herdan_c = 0.0
    stopword_ratio = 0.0
    if n_words:
        c = Counter(words)
        max_rep = max(c.values()) / n_words
        hapax_ratio = sum(1 for _, v in c.items() if v == 1) / n_words
        probs = [v / n_words for v in c.values()]
        ent = -sum(p * math.log(p + 1e-12) for p in probs)
        entropy = ent / (math.log(len(c) + 1e-12) + 1e-6)
        herdan_c = (math.log(len(c) + 1e-12) / (math.log(n_words + 1e-12) + 1e-6)) if n_words > 1 else 0.0
        stopword_ratio = sum(1 for w in words if w in VI_STOPWORDS) / n_words

    bigram_rep = 0.0
    trigram_rep = 0.0
    if n_words >= 2:
        bigrams = list(zip(words, words[1:]))
        bigram_rep = 1.0 - (len(set(bigrams)) / (len(bigrams) + 1e-6))
    if n_words >= 3:
        trigrams = list(zip(words, words[1:], words[2:]))
        trigram_rep = 1.0 - (len(set(trigrams)) / (len(trigrams) + 1e-6))

    compression_ratio = 0.0
    if raw_chars > 0:
        try:
            comp = zlib.compress(raw.encode('utf-8', errors='ignore'), level=9)
            compression_ratio = len(comp) / (raw_chars + 1e-6)
        except Exception:
            compression_ratio = 0.0

    ppl = 0.0
    if mlm_model is not None:
        try:
            ppl = maskedlm_pseudo_ppl(raw)
        except Exception:
            ppl = 0.0

    # Advanced tokenizer-level features (đúng theo cách model nhìn text)
    hf_tokens_trunc = 0
    hf_tokens_full = -1
    hf_is_truncated = 0.0
    hf_subwords_per_word = 0.0
    try:
        enc_trunc = tokenizer(t, add_special_tokens=True, truncation=True, max_length=MAX_SEQ_LEN)
        hf_tokens_trunc = len(enc_trunc.get('input_ids', []) or [])
        hf_subwords_per_word = float(hf_tokens_trunc / (n_words + 1e-6)) if n_words else 0.0
        if chars <= TOKEN_FULL_MAX_CHARS:
            enc_full = tokenizer(t, add_special_tokens=True, truncation=False)
            hf_tokens_full = len(enc_full.get('input_ids', []) or [])
            hf_is_truncated = 1.0 if hf_tokens_full > MAX_SEQ_LEN else 0.0
        else:
            hf_tokens_full = -1
            hf_is_truncated = 1.0 if hf_tokens_trunc >= MAX_SEQ_LEN else 0.0
    except Exception:
        pass

    return {
        'n_chars': float(chars),
        'n_words': float(n_words),
        'n_sents': float(n_sents),
        'n_paras': float(n_paras),

        'punct_ratio': float(punct / (chars + 1e-6)),
        'digit_ratio': float(digits / (chars + 1e-6)),
        'uppercase_ratio': float(uppercase_ratio),
        'quote_ratio': float(quote_count / (raw_chars + 1e-6)),
        'newline_ratio': float(newline_count / (raw_chars + 1e-6)),

        'unique_word_ratio': float(uniq_ratio),
        'avg_word_len': float(avg_word_len),
        'hapax_ratio': float(hapax_ratio),
        'herdan_c': float(herdan_c),
        'entropy_norm': float(entropy),
        'stopword_ratio': float(stopword_ratio),

        'avg_sent_len_words': float(avg_sent_len),
        'avg_para_len_words': float(avg_para_len),
        'burstiness': float(burstiness),
        'bullet_lines': float(bullet_lines),
        'url_count': float(url_count),
        'email_count': float(email_count),

        'max_unigram_rep': float(max_rep),
        'bigram_rep': float(bigram_rep),
        'trigram_rep': float(trigram_rep),
        'compression_ratio': float(compression_ratio),

        'mlm_pseudo_ppl': float(ppl),

        # tokenizer-level
        'hf_tokens_trunc': float(hf_tokens_trunc),
        'hf_tokens_full': float(hf_tokens_full),
        'hf_is_truncated': float(hf_is_truncated),
        'hf_subwords_per_word': float(hf_subwords_per_word),
    }


def feature_ai_score_breakdown(feat: Dict[str, float]) -> Dict[str, float]:
    # Breakdown để bạn kiểm tra từng điểm con + trọng số.
    b = feat.get('burstiness', 0.0)
    u = feat.get('unique_word_ratio', 0.0)
    ent = feat.get('entropy_norm', 0.0)
    r2 = feat.get('bigram_rep', 0.0)
    r3 = feat.get('trigram_rep', 0.0)
    comp = feat.get('compression_ratio', 0.0)
    p = feat.get('punct_ratio', 0.0)
    bullets = feat.get('bullet_lines', 0.0)
    ppl = feat.get('mlm_pseudo_ppl', 0.0)
    spw = feat.get('hf_subwords_per_word', 0.0)
    is_trunc = feat.get('hf_is_truncated', 0.0)

    score_b = 1.0 / (1.0 + math.exp(4.0 * (b - 0.6)))
    score_r = min(max((0.6 * r2 + 0.4 * r3) / 0.30, 0.0), 1.0)
    score_ent = min(max((0.55 - ent) / 0.35, 0.0), 1.0)
    score_u = min(max((0.55 - u) / 0.35, 0.0), 1.0)
    score_c = min(max((1.05 - comp) / 0.45, 0.0), 1.0)
    score_fmt = 1.0 - min(max((p - 0.12) / 0.25, 0.0), 1.0)
    score_bul = 1.0 - min(max(bullets / 8.0, 0.0), 1.0)

    score_ppl = 0.0
    if ppl and ppl > 0:
        score_ppl = min(max((60.0 - ppl) / 50.0, 0.0), 1.0)

    score_spw = min(max((spw - 1.4) / 0.9, 0.0), 1.0)
    score_trunc = float(min(max(is_trunc, 0.0), 1.0))

    # Trọng số mới: tăng nhạy với (burstiness thấp + pseudo-ppl thấp) — kiểu ‘AI viết trơn tru’.
    w_b = 0.32
    w_ppl = 0.20
    w_c = 0.12
    w_fmt = 0.08
    w_bul = 0.03
    w_r = 0.10
    w_ent = 0.05
    w_u = 0.03
    w_spw = 0.05
    w_trunc = 0.02

    raw = (
        w_b * score_b +
        w_r * score_r +
        w_ent * score_ent +
        w_u * score_u +
        w_c * score_c +
        w_fmt * score_fmt +
        w_bul * score_bul +
        w_ppl * score_ppl +
        w_spw * score_spw +
        w_trunc * score_trunc
    )
    raw = float(min(max(raw, 0.0), 1.0))

    return {
        'score_b': float(score_b),
        'score_r': float(score_r),
        'score_ent': float(score_ent),
        'score_u': float(score_u),
        'score_c': float(score_c),
        'score_fmt': float(score_fmt),
        'score_bul': float(score_bul),
        'score_ppl': float(score_ppl),
        'score_spw': float(score_spw),
        'score_trunc': float(score_trunc),
        'w_b': w_b, 'w_r': w_r, 'w_ent': w_ent, 'w_u': w_u, 'w_c': w_c,
        'w_fmt': w_fmt, 'w_bul': w_bul, 'w_ppl': w_ppl, 'w_spw': w_spw, 'w_trunc': w_trunc,
        'feature_score_ai': raw,
    }


def feature_ai_score(feat: Dict[str, float]) -> float:
    # Lưu ý: heuristic chỉ là phụ trợ; nếu muốn ‘phản ánh tốt’ nhất nên học trọng số từ Ai_human.csv.
    return float(feature_ai_score_breakdown(feat).get('feature_score_ai', 0.0))


def combine_scores(prob_ai: float, feat_score: float, w_model: float = 0.6) -> float:
    w = float(min(max(w_model, 0.0), 1.0))
    return float(w * prob_ai + (1.0 - w) * feat_score)


# ---- Final decision (model + features) ----

def ensemble_decision(text: str, clf_model=None, threshold: float = 0.5, w_model: float = 0.6) -> Dict[str, Any]:
    if clf_model is None:
        return {
            'model': None,
            'combined_prob_ai': None,
            'combined_label': None,
            'combined_threshold': float(COMBINED_THRESHOLD),
        }

    # Predict toàn văn (không còn bị “cắt 256 token” như trước)
    clf_entry = detector_predict(text, clf_model, threshold=threshold, agg=DETECTOR_AGG, stride_tokens=CHUNK_STRIDE_TOKENS, max_windows=DETECTOR_MAX_WINDOWS)
    feat = compute_text_features(text)
    feat_score = feature_ai_score(feat)

    prob_model = float(clf_entry['prob_ai'])
    combined = combine_scores(prob_model, feat_score, w_model=w_model)

    return {
        'model': {**clf_entry},
        'combined_prob_ai': float(combined),
        'combined_label': 'AI' if float(combined) >= float(COMBINED_THRESHOLD) else 'Human',
        'combined_threshold': float(COMBINED_THRESHOLD),
    }


COMBINER_DEBIAS_LENGTH = True  # giảm bias theo độ dài của combiner
COMBINER_LENGTH_FEATURES = ['n_chars','n_words','n_sents','n_paras','avg_para_len_words','hf_tokens_full','hf_tokens_trunc','hf_is_truncated','hf_subwords_per_word']

def _combiner_prob_ai(combiner_model, X_row: pd.DataFrame) -> float:


    proba = combiner_model.predict_proba(X_row)[0]

    classes = getattr(combiner_model, 'classes_', None)
    last_est = None
    if hasattr(combiner_model, 'steps') and getattr(combiner_model, 'steps', None):
        last_est = combiner_model.steps[-1][1]
        if classes is None:
            classes = getattr(last_est, 'classes_', None)

    ai_idx = None

    # 1) Label chuỗi
    if classes is not None:
        try:
            cls_list = list(classes)
            for i, c in enumerate(cls_list):
                if isinstance(c, str) and c.strip().lower() == 'ai':
                    ai_idx = i
                    break
            if ai_idx is None:
                for i, c in enumerate(cls_list):
                    if isinstance(c, str) and 'ai' in c.lower():
                        ai_idx = i
                        break
        except Exception:
            ai_idx = None

    # 2) Nhãn số: suy ra bằng dấu coef(prob_model)
    if ai_idx is None:
        try:
            if 'prob_model' in X_row.columns:
                coef_source = last_est if last_est is not None else combiner_model
                if hasattr(coef_source, 'coef_'):
                    pm_idx = list(X_row.columns).index('prob_model')
                    pm_coef = float(coef_source.coef_[0, pm_idx])
                    ai_idx = 1 if pm_coef >= 0 else 0
        except Exception:
            ai_idx = None

    # 3) Fallback: nếu classes có số 1 thì coi đó là AI
    if ai_idx is None and classes is not None:
        try:
            cls_list = list(classes)
            if 1 in cls_list:
                ai_idx = cls_list.index(1)
        except Exception:
            ai_idx = None

    if ai_idx is None:
        ai_idx = 1 if len(proba) > 1 else 0

    return float(proba[ai_idx])

def combiner_top_contrib(combiner_model, X_row: pd.DataFrame, ai_idx: int = 1, top_k: int = 12) -> Optional[pd.DataFrame]:
    try:
        if not hasattr(combiner_model, 'steps'):
            return None
        steps = getattr(combiner_model, 'steps', []) or []
        scaler = None
        for _, est in steps:
            if hasattr(est, 'mean_') and hasattr(est, 'scale_'):
                scaler = est
                break
        if not steps:
            return None
        clf = steps[-1][1]
        if not hasattr(clf, 'coef_'):
            return None
        cols = list(X_row.columns)
        x = X_row.iloc[0].astype(float).values
        if scaler is not None:
            mean = np.asarray(scaler.mean_, dtype=float)
            scale = np.asarray(scaler.scale_, dtype=float)
            scale = np.where(scale == 0, 1.0, scale)
            z = (x - mean) / scale
        else:
            z = x
        coef = np.asarray(clf.coef_, dtype=float)
        if coef.ndim == 2:
            coef = coef[0]
        intercept = float(np.asarray(getattr(clf, 'intercept_', [0.0]), dtype=float).ravel()[0])
        contrib = coef * z
        logit = float(contrib.sum() + intercept)
        if int(ai_idx) == 0:
            contrib = -contrib
            logit = -logit
            intercept = -intercept
        out_coef = coef if int(ai_idx) == 1 else -coef
        df = pd.DataFrame({'feature': cols, 'raw': x, 'scaled': z, 'coef': out_coef, 'contrib': contrib})
        df['abs_contrib'] = df['contrib'].abs()
        df = df.sort_values('abs_contrib', ascending=False)
        head = df.head(int(top_k)).copy()
        summary = pd.DataFrame({
            'feature': ['__INTERCEPT__', '__LOGIT__'],
            'raw': [np.nan, np.nan],
            'scaled': [np.nan, np.nan],
            'coef': [np.nan, np.nan],
            'contrib': [intercept, logit],
            'abs_contrib': [abs(intercept), abs(logit)],
        })
        return pd.concat([summary, head], ignore_index=True)
    except Exception:
        return None

SAMPLE_TEXT = """Tháng trước, chính quyền Tổng thống Donald Trump đã đề xuất một khuôn khổ hòa bình nhằm hướng tới việc chấm dứt xung đột tại Ukraine. Kế hoạch này sau đó được điều chỉnh nhiều lần, bao gồm các nội dung như Ukraine từ bỏ mục tiêu gia nhập NATO và chấp nhận nhượng một số vùng lãnh thổ cho Nga. Đổi lại, Kiev sẽ nhận được các bảo đảm an ninh, tuy nhiên các cam kết này hiện chưa được mô tả chi tiết.

Phát biểu tại Nhà Trắng hôm thứ Hai, khi được hỏi về lý do Ukraine có thể chấp nhận việc mất lãnh thổ, ông Trump cho rằng đây là thực trạng đã xảy ra. Theo ông, những khu vực đó trên thực tế không còn nằm dưới sự kiểm soát của Ukraine. Ông cũng cho biết Mỹ đang tiếp tục làm việc để xây dựng các cơ chế bảo đảm an ninh nhằm hạn chế nguy cơ xung đột tái diễn.

Tổng thống Mỹ cho biết ông đã có các cuộc trao đổi trực tiếp với Tổng thống Nga Vladimir Putin và đánh giá rằng Nga đang thể hiện mong muốn chấm dứt chiến sự. Tuy nhiên, ông nhận định lập trường của cả Nga và Ukraine vẫn có sự thay đổi theo thời điểm, khiến tiến trình đàm phán gặp nhiều khó khăn. Mục tiêu của Mỹ, theo ông Trump, là đưa các bên liên quan về một lập trường chung.

Sau cuộc gặp tại Berlin giữa đặc phái viên Steve Witkoff, ông Jared Kushner và phái đoàn Ukraine, Tổng thống Trump đánh giá các cuộc thảo luận diễn ra theo hướng tích cực, kéo dài và mang tính xây dựng, đồng thời cho rằng tiến trình đàm phán đang có những bước tiến nhất định."""

MODEL_DATASET_SLUG = 'model-check-ai'  # đổi theo tên dataset bạn upload
MODEL_RELATIVE_PATH = pathlib.Path('detector_phobert')
LOCAL_MODEL_PATH = pathlib.Path('detector_phobert')
eval_path = None
if IS_KAGGLE:
    remote_dir = pathlib.Path('/kaggle/input') / MODEL_DATASET_SLUG / MODEL_RELATIVE_PATH
    if remote_dir.exists():
        target_dir = ARTIFACT_DIR / 'detector_phobert'
        target_dir.parent.mkdir(parents=True, exist_ok=True)
        shutil.copytree(remote_dir, target_dir, dirs_exist_ok=True)
        eval_path = target_dir
        print('✔️ Đã copy detector từ', remote_dir)
    else:
        eval_path = ARTIFACT_DIR / 'detector_phobert'
        print('⚠️ Không tìm thấy', remote_dir, '- kiểm tra MODEL_DATASET_SLUG hoặc cấu trúc dataset.')
else:
    print('Đang chạy local nên bỏ qua bước copy từ /kaggle/input.')
    if LOCAL_MODEL_PATH.exists():
        eval_path = LOCAL_MODEL_PATH
        print('✔️ Đang sử dụng detector local tại', LOCAL_MODEL_PATH.resolve())
    else:
        eval_path = ARTIFACT_DIR / 'detector_phobert'
        print('⚠️ Không thấy LOCAL_MODEL_PATH tại', LOCAL_MODEL_PATH.resolve())
        print('   -> Thử load từ thư mục artifacts:', eval_path)

if 'classifier_model' in globals():
    del classifier_model
classifier_model = None

if eval_path and eval_path.exists():
    classifier_model = AutoModelForSequenceClassification.from_pretrained(str(eval_path)).to(DEVICE)
    classifier_model.eval()
    print('✔️ Đã load detector từ', eval_path)

    # Tải MaskedLM để tính pseudo-perplexity (feature)
    _ = load_mlm_model(PPL_MODEL_NAME)

    # ---- Output: ưu tiên 1 kết quả 'đã học' (combiner), fallback heuristic nếu không có ----
    COMBINER_ONLY = True  # theo yêu cầu: chỉ lấy 1 cái đã học
    PRINT_FEATURE_BREAKDOWN = False

    decision = None
    learned = None

    USE_COMBINER_IF_AVAILABLE = True
    if USE_COMBINER_IF_AVAILABLE:
        try:
            import joblib
            import warnings
            warnings.filterwarnings('ignore', message='Trying to unpickle estimator .*')

            def _find_combiner_joblib() -> Optional[pathlib.Path]:
                candidates = []
                candidates += [ROOT / 'combiner_logreg.joblib', ARTIFACT_DIR / 'combiner_logreg.joblib', pathlib.Path.cwd() / 'combiner_logreg.joblib']
                if IS_KAGGLE:
                    inp = pathlib.Path('/kaggle/input')
                    if inp.exists():
                        candidates += list(inp.rglob('combiner_logreg.joblib'))
                for p in candidates:
                    try:
                        pp = pathlib.Path(p)
                        if pp.exists():
                            return pp
                    except Exception:
                        continue
                return None

            comb_path = _find_combiner_joblib()
            if comb_path is not None:
                bundle = joblib.load(comb_path)
                combiner = bundle.get('model', bundle) if isinstance(bundle, dict) else bundle
                comb_cols = bundle.get('cols', None) if isinstance(bundle, dict) else None
                if comb_cols:
                    def _ensemble_decision_with_combiner(text: str) -> Dict[str, Any]:
                        clf_entry = detector_predict(text, classifier_model, threshold=0.5, agg=DETECTOR_AGG, stride_tokens=CHUNK_STRIDE_TOKENS, max_windows=DETECTOR_MAX_WINDOWS)
                        prob_model = float(clf_entry['prob_ai'])
                        feat = compute_text_features(text)
                        X_row = pd.DataFrame([{**feat, 'prob_model': prob_model}]).reindex(columns=comb_cols, fill_value=0.0)
                        if COMBINER_DEBIAS_LENGTH:
                            try:
                                scaler = None
                                if hasattr(combiner, 'steps'):
                                    for _, est in (getattr(combiner, 'steps', []) or []):
                                        if hasattr(est, 'mean_') and hasattr(est, 'scale_'):
                                            scaler = est
                                            break
                                if scaler is not None and hasattr(scaler, 'mean_'):
                                    means = dict(zip(list(comb_cols), list(np.asarray(scaler.mean_, dtype=float).ravel())))
                                    for f in COMBINER_LENGTH_FEATURES:
                                        if f in X_row.columns and f in means:
                                            X_row.at[0, f] = float(means[f])
                                else:
                                    for f in COMBINER_LENGTH_FEATURES:
                                        if f in X_row.columns:
                                            X_row.at[0, f] = 0.0
                            except Exception:
                                pass
                        combined = _combiner_prob_ai(combiner, X_row)
                        return {
                            'model': {**clf_entry},
                            'combined_prob_ai': float(combined),
                            'combined_label': 'AI' if float(combined) >= float(COMBINED_THRESHOLD) else 'Human',
                            'combined_threshold': float(COMBINED_THRESHOLD),
                        }
                    learned = _ensemble_decision_with_combiner(SAMPLE_TEXT)
        except Exception:
            learned = None

    if learned is not None:
        print(json.dumps(learned, ensure_ascii=False, indent=2))
    else:
        decision = ensemble_decision(SAMPLE_TEXT, classifier_model, w_model=0.5)
        print(json.dumps(decision, ensure_ascii=False, indent=2))
        if PRINT_FEATURE_BREAKDOWN:
            print('---- feature_ai_score_breakdown ----')
            feat_dbg = compute_text_features(SAMPLE_TEXT)
            print(json.dumps(feature_ai_score_breakdown(feat_dbg), ensure_ascii=False, indent=2))
else:
    print('⚠️ Không tìm thấy detector_phobert tại', eval_path if eval_path else ARTIFACT_DIR / 'detector_phobert')


Đang chạy local nên bỏ qua bước copy từ /kaggle/input.
✔️ Đang sử dụng detector local tại D:\Đồ án tốt nghiệp\CheckAI\detector_phobert
✔️ Đã load detector từ detector_phobert
{
  "model": {
    "prob_ai": 0.5000000008172898,
    "label": "AI",
    "threshold": 0.5,
    "n_windows": 2,
    "prob_ai_mean": 0.5000000008172898,
    "prob_ai_max": 1.0
  },
  "combined_prob_ai": 0.8848662199924868,
  "combined_label": "AI",
  "combined_threshold": 0.5
}
